<a href="https://colab.research.google.com/github/choarauc/form_ch/blob/main/solution_api_python_snowflake_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Le dojo d'aujourd'hui va vous apprendre à 

> + utiliser une API 
>
>  + voici la [doc](https://randomuser.me/documentation) de l'api  
 + et le [lien](https://randomuser.me/api/?results=10) pour récupérer la data en json 
>
> + manipuler les données en Python 
>
> + les injecter dans Snowflake qui est l'un des DWH le plus utilisé sur le marché
>
>  + tout savoir [snowflake](https://www.lebigdata.fr/snowflake-tout-savoir)  
>>
>> Pour ce faire vous devrez créer un compte sur https://www.snowflake.com/fr/ qui est gratuit pendant 30 jours (aucunes cb n'est requise) 
>>
>> à l'issue du dojo nous supprimerons le compte ! 




# Importer les données de l'API et les stocker dans un df 

In [ ]:
import requests
import pandas as pd

url = 'https://randomuser.me/api/?results=10'

try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    df = pd.DataFrame(data['results'])
    display(df)
except requests.exceptions.RequestException as e:
    print('Une erreur est survenue lors de l\'appel à l\'API :', e)

In [ ]:
df.info()

# Supprimer les colonne dob, registered, cell, id, picture & login

In [4]:
df = df.drop(['dob', 'registered', 'cell', 'id', 'picture', 'login'], axis=1)

In [ ]:
df

# nous voulons récupérer le nom et prénom uniquement puis supprimer la colonne name 

In [ ]:
#nb si vous avez utiliser la fonction normalize json vous n'avez pas à faire ces 2 questions 

In [14]:
df[['first_name', 'last_name']] = df['name'].apply(lambda x: pd.Series([x['first'], x['last']]))

In [16]:
df = df.drop('name', axis=1)

In [ ]:
df

# Idem nous ne garderons que ville et pays puis supprimer location


In [19]:
df[['city', 'country']] = df['location'].apply(lambda x: pd.Series([x['city'], x['country']]))

In [21]:
df = df.drop('location', axis=1)

In [ ]:
df

# Nous allons remettre en ordre les colonnes pour avoir :
 `'gender', 'first_name', 'last_name', 'email', 'phone', 'city', 'country'`

In [24]:
new_order = ['gender', 'first_name', 'last_name', 'email', 'phone', 'city', 'country']
df = df.reindex(columns=new_order)

In [ ]:
df

# A partir de là, nous souhaitons intégrer ces données dans snowflake avec le connecteur 

In [ ]:
!pip install snowflake-connector-python

In [31]:
import snowflake.connector

In [ ]:
cnx = snowflake.connector.connect(
    user='<your_user>',
    password='<your_password>',
    account='<your_account>',
    warehouse='<your_warehouse>',
    database='<your_database>',
    schema='<your_schema>'
)

cur = cnx.cursor()

In [ ]:
marequete = ''
for index, row in df.iterrows():
    marequete = marequete + "('{}', '{}', '{}', '{}', '{}', '{}', '{}'),".format(
        row['gender'].replace("'", "''"),
        row['first_name'].replace("'", "''"),
        row['last_name'].replace("'", "''"),
        row['email'].replace("'", "''"),
        row['phone'].replace("'", "''"),
        row['city'].replace("'", "''"),
        row['country'].replace("'", "''"),
    )

marequete = marequete[:-1]
marequete = "INSERT INTO CLIENT (gender, first_name, last_name, email, phone, city, country) VALUES " + marequete
cur.execute(marequete)

# Valider les modifications
cnx.commit()

In [ ]:
cnx.close()

# Réaliser une viz sur looker depuis snowflake 